Data Cleaning

In [4]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')


In [5]:
def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [20]:
df = pd.read_csv("/home/gkashish/PycharmProjects/Miner/data/training.1600000.processed.noemoticon.csv", header=None,
                 usecols=[0, 5], names=['sentiment', 'text'], encoding="latin1")
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})
df.head()


,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [25]:
%%time
print("Cleaning the tweets...\n")
clean_tweet_texts = []
for i in range(0, len(df)):
    if ((i + 1) % 100000 == 0):
        print("Tweets %d of %d has been processed" % (i + 1, len(df)))
    clean_tweet_texts.append(tweet_cleaner_updated(df['text'][i]))


Cleaning the tweets...



Tweets 100000 of 1600000 has been processed


Tweets 200000 of 1600000 has been processed


Tweets 300000 of 1600000 has been processed


Tweets 400000 of 1600000 has been processed


...

Tweets 1200000 of 1600000 has been processed


Tweets 1300000 of 1600000 has been processed


Tweets 1400000 of 1600000 has been processed


Tweets 1500000 of 1600000 has been processed


Tweets 1600000 of 1600000 has been processed
CPU times: user 7min 18s, sys: 6.28 s, total: 7min 24s
Wall time: 7min 24s


In [27]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['target'] = df.sentiment
clean_df.to_csv('/home/gkashish/PycharmProjects/Miner/data/clean_tweet.csv',encoding='utf-8')

In [41]:
csv = '/home/gkashish/PycharmProjects/Miner/data/clean_tweet_f.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,text,target
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,dived many times for the ball managed to save ...,0
3,my whole body feels itchy and like its on fire,0
4,no it not behaving at all mad why am here beca...,0


In [43]:

my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596041 entries, 0 to 1596040
Data columns (total 2 columns):
text      1596041 non-null object
target    1596041 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [44]:
my_df.to_csv('/home/gkashish/PycharmProjects/Miner/data/clean_tweet_no_null.csv',encoding='utf-8')


In [45]:
np.sum(my_df.isnull().any(axis=1))


0

In [46]:
my_df.isnull().any(axis=0)

text      False
target    False
dtype: bool